# Projectile Motion Data Management

## Data Wrangling with Pandas

In [ ]:
import numpy as np
import pandas as pd


def clean_data(df):
    # Rename columns to snake case
    df.columns = [
        "trial_id",
        "run_date",
        "angle",
        "initial_velocity_units",
        "air_density_kg_m3",
        "c_d",
        "g",
        "theoretical_range_m",
        "measured_range_m",
        "theoretical_height_m",
        "measured_height_m",
        "theoretical_time_s",
        "measured_time_s",
        "lab_section",
        "notes",
    ]
    # Rename run_date to run_timestamp and standardize formats
    df["run_timestamp"] = (
        pd.to_datetime(df["run_date"], format="%Y-%m-%d", errors="coerce")
        .fillna(pd.to_datetime(df["run_date"], format="%m/%d/%Y", errors="coerce"))
        .fillna(pd.to_datetime(df["run_date"], format="%d-%b-%Y", errors="coerce"))
        .fillna(pd.to_datetime(df["run_date"], format="%Y/%m/%d %H:%M", errors="coerce"))
        .fillna(pd.to_datetime(df["run_date"], format="%b %d, %Y", errors="coerce"))
    )
    df = df.drop(columns=["run_date"])
    # Rename angle to angle_deg and convert radians to degrees
    df["angle_deg"] = df["angle"].apply(
        lambda x: np.degrees(float(x.split()[0])) if isinstance(x, str) and "rad" in x else x
    )
    df = df.drop(columns=["angle"])
    # Strip all characters and symbols in angle_deg
    df["angle_deg"] = df["angle_deg"].astype(str).str.extract("(\d+\.?\d*)").astype(float)
    # Round angle_deg to 2 decimal places and parse as float
    df["angle_deg"] = df["angle_deg"].round(2).astype(float)
    # Rename initial_velocity_units to initial_velocity_mps and convert km/h and kmh to m/s
    df["initial_velocity_mps"] = df["initial_velocity_units"].str.extract(r"(\d+\.?\d*)").astype(
        float
    ) * (1000 / 3600)
    df["initial_velocity_mps"] = df["initial_velocity_mps"].where(
        df["initial_velocity_units"].str.contains(r"km/h|kmh", na=False),
        df["initial_velocity_units"],
    )
    df = df.drop(columns=["initial_velocity_units"])
    # Strip all non-numeric characters from initial_velocity_mps
    df["initial_velocity_mps"] = (
        df["initial_velocity_mps"].astype(str).str.extract(r"([-+]?\d*\.?\d+)")[0].astype(float)
    )
    # Round initial_velocity_mps to 2 decimal places
    df["initial_velocity_mps"] = df["initial_velocity_mps"].round(2)
    # Replace ',' with '.' in air_density_kg_m3, handle non-numeric values, and parse as float
    df["air_density_kg_m3"] = (
        df["air_density_kg_m3"]
        .str.replace(",", ".", regex=False)  # Replace ',' with '.'
        .replace("—", None)  # Replace invalid entries with None
        .astype(float)  # Convert to float
    )
    # Parse c_d as a float
    df["c_d"] = pd.to_numeric(df["c_d"], errors="coerce")
    # Set all values in column 'g' to 9.81 as float
    df["g"] = 9.81
    # Remove all units and symbols from theoretical_range_m and measured_range_m, and parse as float
    df["theoretical_range_m"] = pd.to_numeric(
        df["theoretical_range_m"].str.extract(r"([\d.]+)")[0], errors="coerce"
    )
    df["measured_range_m"] = pd.to_numeric(
        df["measured_range_m"].str.extract(r"([\d.]+)")[0], errors="coerce"
    )
    # Remove units and symbols, parse as float
    df["theoretical_height_m"] = df["theoretical_height_m"].str.extract(r"([\d.]+)").astype(float)
    df["measured_height_m"] = df["measured_height_m"].str.extract(r"([\d.]+)").astype(float)
    # Remove units and symbols, parse as float
    df["theoretical_time_s"] = df["theoretical_time_s"].str.extract(r"([\d.]+)").astype(float)
    df["measured_time_s"] = df["measured_time_s"].str.extract(r"([\d.]+)").astype(float)
    # Remove rows where 'trial_id' contains '_dup'
    df = df[~df["trial_id"].str.contains("_dup", na=False)]
    # Remove exact duplicate rows
    df = df.drop_duplicates()
    # Calculate percent error for range, height, and time
    df["percent_error_range"] = (
        abs(df["measured_range_m"] - df["theoretical_range_m"]) / df["theoretical_range_m"] * 100
    )
    df["percent_error_height"] = (
        abs(df["measured_height_m"] - df["theoretical_height_m"]) / df["theoretical_height_m"] * 100
    )
    df["percent_error_time"] = (
        abs(df["measured_time_s"] - df["theoretical_time_s"]) / df["theoretical_time_s"] * 100
    )
    # Remove rows with >= 10% error
    df = df[(df["percent_error_range"] < 10) | (df["percent_error_range"].isnull())]
    # Remove columns with missing values in specified fields
    columns_to_check = [
        "air_density_kg_m3",
        "c_d",
        "theoretical_range_m",
        "measured_range_m",
        "theoretical_height_m",
        "measured_height_m",
        "theoretical_time_s",
        "measured_time_s",
        "run_timestamp",
        "angle_deg",
        "initial_velocity_mps",
        "percent_error_range",
        "percent_error_height",
        "percent_error_time",
    ]
    df = df.dropna(subset=columns_to_check)
    # Rename columns for height, range, and time
    df = df.rename(
        columns={
            "theoretical_height_m": "height_theor_m",
            "measured_height_m": "height_meas_m",
            "theoretical_range_m": "range_theor_m",
            "measured_range_m": "range_meas_m",
            "theoretical_time_s": "time_theor_s",
            "measured_time_s": "time_meas_s",
        }
    )
    # Rename columns for percent error, range, and time
    df = df.rename(
        columns={
            "percent_error_height": "pct_err_height",
            "percent_error_range": "pct_err_range",
            "percent_error_time": "pct_err_time",
        }
    )
    # Rename columns for clarity
    df = df.rename(
        columns={
            "lab_section": "section",
            "trial_id": "id",
            "run_timestamp": "timestamp",
            "initial_velocity_mps": "vel_init_mps",
            "air_density_kg_m3": "air_density_kgpm3",
            "c_d": "drag_coeff",
        }
    )
    # Sort DataFrame by 'id' column
    df = df.sort_values(by="id")
    return df


# Loaded variable 'df' from URI: ../../data/raw/projectile-motion_practice.csv
df = pd.read_csv("../../data/raw/projectile-motion_practice.csv", engine="pyarrow")

df_clean = clean_data(df.copy())
df_clean.head()


## Write Cleaned Data to Files

In [ ]:
out_path_csv = "../../data/processed/projectile-motion_practice_clean.csv"
out_path_parquet = "../../data/processed/projectile-motion_practice_clean.parquet"
df_clean.to_csv(out_path_csv, index=False)
df_clean.to_parquet(out_path_parquet, index=False)
